In [2]:
#!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 22.2 MB/s 
     |████████████████████████████████| 240 kB 50.7 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=f48e079efe08e6281c14242d337f781bea885d8e2c6b4235d477aa11665f4d6e
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from torch.utils.data import Dataset as td
from sklearn.metrics import accuracy_score
import pandas as pd

import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,lemma,pos')
from collections import defaultdict, Counter

import nltk
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words("russian"))

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Задания 1, 2: категория и тональность упоминаний

Выделяем только аспекты, встретившиеся в train'е, приписываем самую частотную категорию. 

In [4]:
train_asp = pd.read_csv('train_aspects.txt', 
                        delimiter='\t', 
                        names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
                        
)
train_texts = pd.read_csv('train_reviews.txt', 
                          delimiter='\t', 
                          names=['text_id','text'])

In [5]:
def normalize(text):
    doc = nlp(text)
    words = [word.lemma for sent in doc.sentences for word in sent.words]
    return words

In [ ]:
train_asp['norm_mention'] = [tuple(normalize(m)) for m in train_asp['mention']]

Строим частотный словарь "токенизированное упоминание + категория"

Категория - аспектная категория или тональность

In [ ]:
def get_mention_category(data, cat_type):
    mention_categories = data.value_counts(subset=['norm_mention', cat_type])
    mention_categories_dict = defaultdict(dict)
    for key, value in mention_categories.items():
        mention_categories_dict[key[0]][key[1]] = value
 
    return {k: Counter(v).most_common(1)[0][0] for k, v in mention_categories_dict.items() if int(Counter(v).most_common(1)[0][1]) >= 6}

In [ ]:
best_mention_cat = get_mention_category(train_asp, 'category')

In [ ]:
best_mention_sentiment = get_mention_category(train_asp, 'sentiment')

In [ ]:
upd_best_mention_sentiment = {}
for k, v in best_mention_sentiment.items():
    if k in best_mention_cat.keys():
        upd_best_mention_sentiment[k] = v


upd_best_mention_cat = {}
for k, v in best_mention_cat.items():
    if k in upd_best_mention_sentiment.keys():
        upd_best_mention_cat[k] = v

In [ ]:
test_texts = pd.read_csv('test_reviews.txt', delimiter='\t', names=['text_id', 'text'])

Длины упоминаний аспектов в трейне:

In [ ]:
Counter([len(x) for x in upd_best_mention_sentiment.keys()])

Будем учитывать только упоминания длиной 1-5 токенов:

In [ ]:
def label_texts(text, mentions, sentiments, max_len=5):
    tokenized = [word for sent in nlp(text).sentences for word in sent.words]
    text_end = len(tokenized)
    for i, token in enumerate(tokenized):
        for l in reversed(range(max_len)):
            if i + l > text_end:
                continue
            span = tokenized[i:i + l]
            if len(span) != 0:
                if span[0].upos == 'NOUN' or span[0].upos == 'VERB' or len(span) > 1:
                    key = tuple([t.lemma for t in span])
                    if key in mentions:
                        start, end = span[0].start_char, span[-1].end_char
                        yield mentions[key], text[start:end], start, end, sentiments[key]
                        break

Применяем частотные данные к текстам из dev:

In [ ]:
with open('test_pred_aspects.txt', 'w', encoding='utf-8') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for asp in label_texts(text, upd_best_mention_cat, upd_best_mention_sentiment):
            print(idx, *asp, sep="\t", file=f)

In [ ]:
with open('test_pred_aspects.txt', 'r', encoding='utf-8') as f:
    new_final = []
    final = []
    asps = f.readlines()
    real_asps = [a.strip().split('\t') for a in asps]
    for i in range(len(real_asps)-1):
        f_start = real_asps[i][3]
        f_end = real_asps[i][4]
        s_start = real_asps[i+1][3]
        s_end = real_asps[i+1][4]
        if f_start == s_start:
            if f_end > s_end:
                final.append(real_asps[i])
            else:
                final.append(real_asps[i+1])
        elif f_end == s_end:
            if f_start < s_start:
                final.append(real_asps[i])
            else:
                final.append(real_asps[i+1])
        else:
            final.append(real_asps[i])
            final.append(real_asps[i+1])
for f in final:
    ff = '\t'.join(f)
    if ff not in new_final:
        new_final.append(ff)

In [ ]:
with open('test_pred_aspects.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(new_final))

# Задание 3

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 2), lowercase=True, max_features=150000)
ros = RandomOverSampler(random_state=42)

In [ ]:
class Dataset(td):

    def __init__(self, df, column, tokenizer=None):

        self.labels = [label for label in df[column]]
        if tokenizer:
            self.texts = [tokenizer(text, 
                                    padding='max_length', 
                                    max_length = 512, truncation=True,
                                    return_tensors="pt") for text in df['text']]
        else:
            self.texts = [text for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [ ]:
train_df_cats = pd.read_csv("train_cats.txt", sep="\t", names=['id', 'cat', 'pos'])
test_df = pd.read_csv("test_reviews.txt", sep="\t", names=['id', 'text'])

In [ ]:
train_df_cats = train_df_cats[train_df_cats.cat=='Whole']
train_joined = train_texts.join(train_df_cats.set_index('id'), on='id', lsuffix='_caller', rsuffix='_other')
train_joined = train_joined[train_joined.cat == 'Whole'].replace({'positive':2, 'negative':-2, 'both':1, 'neutral':-1, 'absence':0})

In [ ]:
X_train_text = text_transformer.fit_transform(train_joined['text'])
X_test_text = text_transformer.transform(test_df['text'])

In [ ]:
ds = Dataset(train_joined, 'pos')
ds.texts = X_train_text

x, y = ros.fit_resample(ds.texts, ds.labels)
logit = LogisticRegression(C=6e1, solver='newton-cg', multi_class='multinomial', random_state=17, n_jobs=4)
logit.fit(x, y)

test_df['pos'] = logit.predict(X_test_text)

In [ ]:
test_df['pos'] = test_df['pos'].map({2: 'positive',  -2: 'negative', 1: 'both', -1: 'neutral', 0: 'absence'})

In [ ]:
test_df.to_csv('test_pred_cats.txt', index=False)